Testing new TAVERN library with Augustine data used in original NSF proposal submission.

NOTES ON MAGMA PARAMETERS FROM THE LITERATURE

AUGUSTINE WEBSTER ET AL., 2010
------------------------------------------
SAMPLES
For the 2006 eruption we refer to the shallow storage as the Low Silica Andesite (LSA), 
the intermediate storage as High Silica Andesite (HSA) and the deep primitive source is 
inferred but not well constrained.

LOW-SILICA ANDESITE
**Shallow Storage**
T = 900-970 deg C (970 might be the primitive recharge)
fO2 = 
P = 100-200 MPa (4-8 km)

HIGH-SILICA ANDESITE
**Intermediate Storage**
T = 850-880 deg C
fO2 = 
P = 130-160 MPa

DEEP PRIMITIVE MAGMA
**Deep Storage**
T = 970-? deg C
fO2 = 
P = 

GENERAL
The magmatic fluids were relatively oxidizing and included H2O-enriched 
S-, S2-, and SO2+/- CO2-bearing vapors; hydrosa-line aqueous 
liquids largely enriched in Cl-, SO42-, alkalis, and H2O; and moderately 
saline, H2O-poor liquids containing Cl-, SO42-, and alkali elements.

OXYGEN FUGACITY
Coexisting iron-titanium
oxides of 2006 rock samples, which are generally consistent
with those of prior eruptive materials, indicate fO2
values of approximately NNO+1.5 to NNO+2.5 and oxide crystalliza-
tion temperatures of 835 to 1,052 degC.

SULFUR
As sulfides are rare and only present in Augustine rocks
at abundance levels well below that required to remove
significant S from the residual melts and because magma
mixing is not solely responsible for the strong reduction in
S concentrations with melt evolution, the involvement of
a fluid or fluid phases in S sequestration is required.

In [1]:
import tavern as tv

## Define melt compositions

In [2]:
#Low-silica andesite groundmass from Larsen et al. 2010
shallow_melt = tv.SilicateMelt({	"SiO2": 71.16, 
					"TiO2": 0.97, # 06AUMC005c
					"Al2O3": 13.72,
					"FeO": 4.03,
					"Fe2O3": 0.,
					"MnO": 0.12,
					"MgO": 0.90,
					"CaO": 3.13,
					"Na2O": 3.37,
					"K2O": 2.06,
					"P2O5": 0.23,
					"H2O": 2.00, #TODO need better vals
					"CO2": 0.002,
					"S": 0.02
					})


#high-silica andesite
#HSA matrix glass from Larsen et al. 2010
int_melt = tv.SilicateMelt({	"SiO2": 75.86, #Values in wt% 
					"TiO2": 0.44, 
					"Al2O3": 12.81,
					"FeO": 2.08, 
					"Fe2O3": 0.,
					"MnO": 0.08,
					"MgO": 0.41,
					"CaO": 2.09,
					"Na2O": 3.78,
					"K2O": 0.99,
					"P2O5": 0.14,
					"H2O": 4.10, #Highest reported evolved augustine MI in Webster et al 2010
					"CO2": 0.011, #Highest reported evolved augustine MI in Webster et al 2010
					"S": 0.02
					})


#primitive mafic recharge
deep_melt = tv.SilicateMelt({	"SiO2": 54.92, #Values in wt% #Augustine basalt sample from Webster et al 2010 (cpx-hosted MI)
					"TiO2": 0.62, #RB-W91A137 cpx MI
					"Al2O3": 14.62,
					"FeO": 6.86, 
					"Fe2O3": 0.,
					"MnO": 0.16,
					"MgO": 5.89,
					"CaO": 10.33,
					"Na2O": 2.04,
					"K2O": 0.61,
					"P2O5": 0.15,
					"H2O": 4.00, #Water by difference from reheated MI, max value
					"CO2": 0.02, #TODO... this is a WAG
					"S": 0.23
					})

## Define magma conditions

In [3]:
#SHALLOW
shallow_XCO2 = 0.10
shallow_temp = 900.0
shallow_press = 1000.0
shallow_fO2_buffer = 'QFM'
shallow_fO2_delta = 1.0

#INT
int_XCO2 = 0.10
int_temp = 850.0
int_press = 1500.0
int_fO2_buffer = 'QFM'
int_fO2_delta = 1.0

#DEEP
deep_XCO2 = 0.10
deep_temp = 970.0
deep_press = 3000.0
deep_fO2_buffer = 'QFM'
deep_fO2_delta = 1.0

## Define surface gas composition

In [4]:
surface_gas = tv.MagmaticFluid({"H2O": 94.08,
                                "CO2": 3.36,
                                "SO2": 3.34},
                               units="molpercent")

## Calculate equilibrium fluids

In [5]:
# First get fH2O from Moore model
shallow_fH2O = tv.calculate_fH2O_from_melt(sample=shallow_melt,
                                           pressure=shallow_press,
                                           temperature=shallow_temp).result

int_fH2O = tv.calculate_fH2O_from_melt(sample=int_melt,
                                       pressure=int_press,
                                       temperature=int_temp).result

deep_fH2O = tv.calculate_fH2O_from_melt(sample=deep_melt,
                                        pressure=deep_press,
                                        temperature=deep_temp).result

In [6]:
# Second create model objects
shallow_mod = tv.Model(pressure=shallow_press, temperature=shallow_temp, fO2_buffer=shallow_fO2_buffer,
                       fO2_delta=shallow_fO2_delta, H2O_param=shallow_fH2O, H2O_param_type='fugacity',
                       SC_param=shallow_XCO2, SC_param_type='CO2molfrac')

int_mod = tv.Model(pressure=int_press, temperature=int_temp, fO2_buffer=int_fO2_buffer,
                       fO2_delta=int_fO2_delta, H2O_param=int_fH2O, H2O_param_type='fugacity',
                       SC_param=int_XCO2, SC_param_type='CO2molfrac')

deep_mod = tv.Model(pressure=deep_press, temperature=deep_temp, fO2_buffer=deep_fO2_buffer,
                       fO2_delta=deep_fO2_delta, H2O_param=deep_fH2O, H2O_param_type='fugacity',
                       SC_param=deep_XCO2, SC_param_type='CO2molfrac')

In [7]:
# Third calculate equilibrium fluid on Model objects
shallow_eq_fluid = shallow_mod.equilibrium_fluid()
int_eq_fluid = int_mod.equilibrium_fluid()
deep_eq_fluid = deep_mod.equilibrium_fluid()

Calculated pressure error is -7.95807864051312e-13 bars
Calculated pressure error is 6.82121026329696e-13 bars


In [8]:
# Fourth, respeciate all calculated fluids at 1 bar for input into mixing model
shallow_eq_lowP = shallow_mod.respeciate(shallow_eq_fluid, pressure=1)
int_eq_lowP = int_mod.respeciate(int_eq_fluid, pressure=1)
deep_eq_lowP = deep_mod.respeciate(deep_eq_fluid, pressure=1)

## Calculate degassed fluids

In [9]:
# First get dissolved volatile composition of all three magmas
shallow_melt_volatiles = shallow_melt.get_dissolved_volatile_composition()
int_melt_volatiles = int_melt.get_dissolved_volatile_composition()
deep_melt_volatiles = deep_melt.get_dissolved_volatile_composition()

# also create a completely degassed sample with values of 0, 0, 0
zero_volatiles = tv.MagmaticFluid({"H2O": 0.0, "CO2": 0.0, "S": 0.0})

In [10]:
# Second calculate degassed fluid compositions
shallow_degassing = tv.calculate_degassed_fluid_composition(deep_volatiles=shallow_melt_volatiles,
                                                            shallow_volatiles=zero_volatiles).result

int_degassing = tv.calculate_degassed_fluid_composition(deep_volatiles=int_melt_volatiles,
                                                        shallow_volatiles=shallow_melt_volatiles).result

deep_degassing = tv.calculate_degassed_fluid_composition(deep_volatiles=deep_melt_volatiles,
                                                         shallow_volatiles=zero_volatiles).result

In [11]:
# Third speciate fluid compositions at 1 bar and conditions of each magma body
shallow_degassing = tv.calculate_speciation(sample=shallow_degassing, pressure=1, temperature=shallow_temp,
                        fO2_buffer=shallow_fO2_buffer, fO2_delta=shallow_fO2_delta).result

int_degassing = tv.calculate_speciation(sample=int_degassing, pressure=1, temperature=int_temp,
                        fO2_buffer=int_fO2_buffer, fO2_delta=int_fO2_delta, opt='leastsq').result

deep_degassing = tv.calculate_speciation(sample=deep_degassing, pressure=1, temperature=deep_temp,
                        fO2_buffer=deep_fO2_buffer, fO2_delta=deep_fO2_delta).result

print(shallow_degassing.get_composition())
print(int_degassing.get_composition())
print(deep_degassing.get_composition())

CO     2.652101e-04
CO2    9.425457e-02
H2     5.915201e-02
H2O    9.427193e+01
H2S    1.223065e-01
S2     5.410031e-03
SO2    5.446686e+00
O2     3.770711e-10
dtype: float64
CO     1.043332e-03
CO2    4.264416e-01
H2     5.914759e-02
H2O    9.951337e+01
H2S    0.000000e+00
S2     0.000000e+00
SO2    0.000000e+00
O2     4.373093e-11
dtype: float64
CO     1.273384e-03
CO2    3.695857e-01
H2     4.749935e-02
H2O    7.403451e+01
H2S    1.630636e-01
S2     3.803031e-02
SO2    2.534604e+01
O2     5.124943e-09
dtype: float64


## Do the matching model

In [12]:
tv.match(sub_gases={"Shallow Degassing": shallow_degassing,
                    "Shallow EQ Fluid": shallow_eq_lowP,
                    "Int Degassing": int_degassing,
                    "Int EQ Fluid": int_eq_lowP,
                    "Deep Degassing": deep_degassing,
                    "Deep EQ Fluid": deep_eq_lowP},
        surface_gas=surface_gas).result

Generating all possible gas combinations...


FileNotFoundError: [Errno 2] No such file or directory: 'sums_6_100_1.p'